# **Generate a timelapse from existing video**  
  
### **Table of Contents**
1. [Set Paths](#paths)
2. [YOLO-Model](#model)
3. [Parameters](#params)
4. [Process video](#process_video)

In [21]:
from ultralytics import YOLO
import os
import cv2
import math

### **Set Paths** <a id="paths"></a>

`video` = path to the raw video of the printing video  
`weights` = best weights for yolo model achieved during training  
`outputFolder` = path to store the created timelapse

In [22]:
video = "C:\\Users\\Fabio\\Desktop\\VideoLeft.mp4"
weights = "runs/detect/train/weights/04.03.25_left.pt"
outputFolder = "C:\\Users\\Fabio\\Desktop\\TestOutput"

---

### **Import trained YOLO model** <a id="model"></a>
* currently using YOLOV8

In [23]:
model = YOLO(weights)

---

### **Set parameters**  <a id="params"></a>
* before processing we need to capture fps, set params for tracking and frame skipping

In [ ]:
cap = cv2.VideoCapture(0)

# get raw vid fps
if cap.get(cv2.CAP_PROP_FPS) == 30:
    fSkip = 7.5
else:
    fSkip = math.ceil(cap.get(cv2.CAP_PROP_FPS)/4)

prevX = 1
lastWasTheSame = False
wiggleRoom = 0.003
frame_skip = fSkip
frame_count = 0
x = None

---

### **Process Video Frame-by-Frame** <a id="process_video"></a>
* The while loop reads the video frame by frame, detects the printhead's position, and saves the necessary images.

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    
    
    
    if not ret:
        break

    # extract dimensions once on first frame
    if frame_count == 0:
        hImg, wImg, _ = frame.shape

    if frame_count % frame_skip == 0:
        results = model.predict(frame)
        frame_skip = fSkip

        for r in results:
            for box in r.boxes:
                x, y, w, h = box.xywh[0].tolist() # Get the x, y, w, h coordinates.
                xNorm, yNorm, wNorm, hNorm =  x/wImg, y/hImg, w/wImg, h/hImg

                if abs(xNorm - prevX) < wiggleRoom:
                    if not lastWasTheSame:
                        lastWasTheSame = True
                    else:
                        lastWasTheSame = False
                        filename = os.path.join(outputFolder, f"frame_{frame_count}.jpg")
                        cv2.imwrite(filename, frame)
                        frame_skip = 100
                        
                if prevX > xNorm:
                    prevX =  xNorm
    frame_count += 1
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()